In [11]:
from langchain_community.document_loaders import WebBaseLoader

In [12]:
urls=["https://www.breedr.co/news/what-are-the-new-farming-techniques"]

loader = WebBaseLoader(urls)
docs = loader.load()
docs

[Document(metadata={'source': 'https://www.breedr.co/news/what-are-the-new-farming-techniques', 'title': 'What are the new farming techniques for 2022 and beyond?', 'description': 'What are the new farming techniques? And how can they make your farm more sustainable? We take a look in this blog.', 'language': 'en'}, page_content="\n\nWhat are the new farming techniques for 2022 and beyond?\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLog In\nShop\nSupport\nContact Us\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\t\t\t\t\t\t\tSolutions\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\n\n\n\n\n\n\n\nBreedr Tools\n\n\n\n\t\t\t\t\t\t\t\t\t\t\t\tLivestock App\n\t\t\t\t\t\t\t\t\t\t\t\n\n\n\n\nFarm System\n\n\n\n\t\t\t\t\t\t\t\t\t\t\t\tSuckler\n\t\t\t\t\t\t\t\t\t\t\t\n\n\n\n\t\t\t\t\t\t\t\t\t\t\t\tFinisher\n\t\t\t\t\t\t\t\t\t\t\t\n\n\n\n\t\t\t\t\t\t\t\t\t\t\t\tRearer\n\t\t\t\t\t\t\t\t\t\t\t\n\n

In [2]:
big_text = """**Vegetable Prices Soar, Protests Erupt in Southern India**

*June 20, 2024 – The Daily Herald*

*Chennai, India* – A wave of protests has swept across several southern Indian states in response to a sharp increase in vegetable prices, adding to the economic woes of millions already grappling with inflation. The recent surge in prices has exacerbated the financial strain on households, leading to widespread demonstrations in cities and rural areas alike.

Over the past month, the cost of essential vegetables such as tomatoes, onions, and green chilies has risen by up to 60%. In Chennai, the price of tomatoes has skyrocketed from ₹30 per kilogram to ₹80 per kilogram, while onions now cost ₹50 per kilogram, up from ₹20. These price hikes have severely impacted daily budgets, especially for low- and middle-income families.

Experts attribute the surge to a combination of factors, including unseasonal rains that have damaged crops, logistical disruptions, and increased transportation costs. The erratic weather patterns, linked to climate change, have led to crop failures in key agricultural regions, further straining the supply chain.

As prices continue to climb, thousands of farmers, traders, and consumers have taken to the streets in protest. In Tamil Nadu, Karnataka, Kerala, and Andhra Pradesh, demonstrators have blocked highways, staged sit-ins, and organized marches, demanding immediate government intervention to stabilize prices and provide relief.

In Chennai, the capital of Tamil Nadu, protesters gathered at Marina Beach, holding placards and chanting slogans against the state and central governments. "The government must take swift action to control prices. We are struggling to feed our families," said Lakshmi, a homemaker who joined the protests.

Farmers, too, are feeling the pinch. "Our crops have been ruined by the unpredictable weather, and now we are being blamed for the high prices," said Rajendran, a farmer from Karnataka. "We need support, not blame."

The political response has been swift but varied. State governments have announced measures such as importing vegetables from other states and countries, subsidizing essential items, and setting up fair price shops. However, these steps have so far done little to alleviate the immediate pressure on consumers.

The central government has also come under fire for its handling of the situation. Opposition parties have accused the ruling administration of neglecting the agricultural sector and failing to address the root causes of inflation. "The government must take responsibility and act now to protect the livelihoods of millions," said Rahul Gandhi, leader of the Indian National Congress.

Economists warn that without long-term solutions, such as improving agricultural infrastructure and adopting more sustainable farming practices, the volatility in vegetable prices is likely to continue. "We need a comprehensive approach that addresses climate resilience, supply chain efficiency, and market regulation," said Dr. Arvind Subramanian, a leading agricultural economist.

As the protests continue, there is growing pressure on authorities to find a resolution. For now, the soaring vegetable prices remain a stark reminder of the broader economic challenges facing the country.

The Daily Herald will continue to monitor the situation and provide updates as more information becomes available.

---

*For more news and updates, follow The Daily Herald on Twitter @DailyHeraldIndia and visit our website at www.dailyheraldindia.com.*"""

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader , DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [14]:
# Initialize text splitter
txt_splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=400
)

# Split the loaded documents
inp_txt = txt_splitter.split_documents(docs)

print("Data Chunks Created")

Data Chunks Created


In [15]:
hfembeddings = HuggingFaceEmbeddings(
    model_name = "thenlper/gte-large",
    model_kwargs = {'device':'cuda'}
)
print("Embedding Created")

Embedding Created


In [16]:
from langchain_community.vectorstores import FAISS
db = FAISS.from_documents(inp_txt,hfembeddings)
db.save_local(r"P:/college stuffs/mini project/Agri-Hub - Copy/datas/faiss/newweb-data/")
print("Vector Store Creation Completed and Stored Locally")

Vector Store Creation Completed and Stored Locally


In [7]:
from langchain import PromptTemplate
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import CTransformers , HuggingFaceHub
from langchain.chains import RetrievalQA
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [8]:
PROMPT_TEMPLATE = '''
With the information provided try to answer the question. 
If you cant answer the question based on the information either say you cant find an answer or unable to find an answer.
So try to understand in depth about the context and answer only based on the information provided. Dont generate irrelevant answers

Context: {context}
Question: {question}
Do provide only helpful answers

Helpful answer:
'''
INP_VARS = ['context', 'question']
custom_prompt_template = PromptTemplate(
    template = PROMPT_TEMPLATE,
    input_variables = INP_VARS
)

In [9]:
llm = CTransformers(
    model = r"D:\agribot\llama-2-7b-chat.ggmlv3.q4_1.bin",
    model_type="llama",
    max_new_tokens = 512,
    temperature = 0.2,
    callbacks=[StreamingStdOutCallbackHandler()],
    gpu_layers=0
)

In [10]:
hfembeddings = HuggingFaceEmbeddings(
    model_name = "thenlper/gte-large",
    model_kwargs = {'device':'cuda'}
)
print("Embedding Created")

d:\newenv\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Embedding Created


In [11]:
vector_db = FAISS.load_local(r"D:\agribot\faiss\agri_web_data/", hfembeddings,allow_dangerous_deserialization=True)

In [12]:
retrieval_qa_chain = RetrievalQA.from_chain_type(
                                llm=llm,
                                chain_type="stuff",
                                retriever=vector_db.as_retriever(search_kwargs={'k': 1}),
                                return_source_documents=True,
                                chain_type_kwargs={"prompt": custom_prompt_template}
                            )

In [14]:
user_input = "What scopes of this journel?"
prompt = {'query': user_input}
model_out = retrieval_qa_chain(prompt)
answer = model_out['result']
print(answer)

The scope of this journal includes but is not limited to:
1. Research on the impact of technology on society and culture
2. Studies on the ethical implications of emerging technologies
3. Analysis of the social, economic, and political aspects of technology
4. Discussions on the role of technology in shaping the future of work and education
5. Reviews of new and emerging technologies and their potential applications
6. Critical examinations of the intersection of technology and other fields such as philosophy, psychology, and sociology
7. Book reviews, literature reviews, and theoretical frameworks related to the field of technology and society
8. Opinion pieces, editorials, and commentaries on current events and trends in the field
9. Interviews with leading scholars, researchers, and innovators in the field.

I cant answer this question because there is no clear answer from the information provided. The scope of the journal is not specified by category and also include discussion for